In [20]:
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn import metrics
# from KaggleWord2VecUtility import KaggleWord2VecUtility
import pandas as pd
import numpy as np

In [2]:
#train = pd.read_csv("./data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
#test = pd.read_csv("./data/testData.tsv", header=0, delimiter="\t", quoting=3)
train = pd.read_csv("./target-sentiment-analysis/train.tsv", header=None, delimiter="\t", quoting=3)
test = pd.read_csv("./target-sentiment-analysis/test.tsv", header=None, delimiter="\t", quoting=3)
#print("The first review is: ")
#print(train["review"][0])

print(train[2][0])
#print(train[0][2])

13


In [3]:
from bs4 import BeautifulSoup

example1 = BeautifulSoup(train[5][0])
print(example1.get_text())

Gas by my house hit $3.39!!!! I'm going to Chapel Hill on Sat. :)


In [4]:
import re

letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      example1.get_text() )  # The text to search
print(letters_only)

Gas by my house hit           I m going to Chapel Hill on Sat    


In [5]:
lower_case = letters_only.lower()        # Convert to lower case
words = lower_case.split()               # Split into words

In [6]:
import nltk
from nltk.corpus import stopwords

print(stopwords.words("english"))

ModuleNotFoundError: No module named 'nltk'

In [8]:
words = [w for w in words if not w in stopwords.words("english")]
print(words)

['gas', 'house', 'hit', 'going', 'chapel', 'hill', 'sat']


In [6]:
def review_to_words(raw_review, index1, index2):
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    #
    # 3. Convert to lower case, split into individual words
    words = review_text.lower().split()                             
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    if index1 == index2:
        if index1 >= len(words):
            return words[len(words)-1]
        return words[index1]
    else:
        temp = words[index1:index2]
        return( " ".join( temp )) 

In [7]:
clean_review = review_to_words( train[5][107], train[2][107], train[3][107] )
print(clean_review)

:-/


In [8]:
# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range(0, len(train[5])):
    clean_train_reviews.append(review_to_words( train[5][i], train[2][i], train[3][i]))

print(clean_train_reviews)

[':)', "can't", 'hopefully', 'make', 'successful', 'tough', 'stresses me', 'have', 'superbowl', "ain't", 'not with', 'just', ';)', 'didnt', 'shes', 'release', 'maybe', 'delisting', 'good', 'fantastic', 'left', 'miss the rematch', 'game', 'up!', 'turn your back on', 'so', 'dreams and', 'means', 'act', 'was a win for me.', 'too much', "can't", 'big ass get on my', 'not so', 'omg', 'good', 'not', 'thank', "candy....wasn't even the good", 'warning', 'better', 'missing', 'perf', 'great', 'plenty', 'plenty of room. pretty fired', 'plenty of ups &', 'hope', 'nooooooooo', "can't", ":'(", 'may need', 'fave', 'sweet.', 'good', "may have a heart attack for jason wu's new", 'correctly', 'amazing concert', 'hope to see you there', 'lmfao', 'good', 'well known', 'overhaul:', 'damn', 'shit', 'fails', 'divides', 'moving', '#untouchable no', 'hopefully', 'pumped', 'eased', 'pushed', 'managed to', "doesn't", 'hopefully', 'decent', 'available', 'great', 'hate the', 'just', 'just', 'fine', ':)', 'fucking'

In [9]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()
print(train_data_features)

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [10]:
vocab = vectorizer.get_feature_names()
print(vocab)

['0g5baw47', '11', '11th', '15', '1love', '1st', '21st', '24th', '27', '2nd', '30th', '3rd', '4ever', '4th', '50', '5th', '6th', '778', '7pm', '7thingsaboutmybestfriend', '90', '__', '___', 'aaaahhhh', 'aaawwh', 'abandon', 'abandoned', 'abandonment', 'ability', 'able', 'about', 'absconding', 'absence', 'absolute', 'absolutely', 'abused', 'accelerate', 'acceptable', 'accepted', 'accessible', 'accidentally', 'accidents', 'accomplished', 'accuse', 'ace', 'achievement', 'achilles', 'achy', 'acknowledged', 'act', 'actually', 'adamant', 'add', 'added', 'addicted', 'addictive', 'adding', 'admire', 'admit', 'adorable', 'adore', 'advance', 'advanced', 'advantage', 'affected', 'affecting', 'affectionate', 'afford', 'afraid', 'after', 'again', 'against', 'agaist', 'agree', 'agreed', 'agreement', 'ah', 'ahaha', 'ahahahaha', 'ahead', 'ahh', 'ahhaha', 'ahhhh', 'ahhhhh', 'aimed', 'aims', 'ain', 'aint', 'aintgottimeforthatshit', 'air', 'airport', 'alike', 'alive', 'all', 'allegedly', 'allowed', 'alls'

In [11]:
dist = np.sum(train_data_features, axis=0)
for tag, count in zip(vocab, dist):
    print(count, tag)

1 0g5baw47
1 11
1 11th
1 15
1 1love
12 1st
1 21st
1 24th
1 27
8 2nd
1 30th
1 3rd
1 4ever
2 4th
1 50
1 5th
1 6th
1 778
1 7pm
1 7thingsaboutmybestfriend
1 90
1 __
1 ___
1 aaaahhhh
1 aaawwh
1 abandon
2 abandoned
2 abandonment
1 ability
4 able
13 about
1 absconding
1 absence
3 absolute
10 absolutely
1 abused
1 accelerate
1 acceptable
3 accepted
1 accessible
2 accidentally
1 accidents
2 accomplished
1 accuse
2 ace
1 achievement
1 achilles
1 achy
1 acknowledged
2 act
5 actually
1 adamant
1 add
1 added
2 addicted
1 addictive
1 adding
1 admire
1 admit
4 adorable
1 adore
1 advance
1 advanced
1 advantage
2 affected
1 affecting
1 affectionate
2 afford
1 afraid
1 after
5 again
5 against
1 agaist
3 agree
2 agreed
1 agreement
3 ah
1 ahaha
1 ahahahaha
1 ahead
2 ahh
1 ahhaha
2 ahhhh
1 ahhhhh
1 aimed
1 aims
7 ain
3 aint
1 aintgottimeforthatshit
1 air
1 airport
1 alike
1 alive
34 all
1 allegedly
1 allowed
1 alls
6 almost
2 alone
2 alot
9 already
5 alright
8 also
1 although
27 always
12 am
1 amaaaze
1 am

1 luke
1 lukewarm
1 lurve
1 luv
1 lying
10 mad
12 made
5 madness
1 madonna
1 magic
1 magical
2 main
1 major
44 make
1 makeithappen
13 makes
3 making
1 mama
2 man
2 managed
8 many
1 marvellous
1 massage
1 massive
1 matches
1 matey
1 matters
1 mauled
1 max
176 may
3 mayb
12 maybe
1 mayor
1 mbaaah
1 mcknight
45 me
3 means
2 media
3 meet
1 mega
1 mellow
1 memories
1 mental
1 merica
4 merry
2 mess
2 messed
1 mexico
1 mid
24 might
3 mighty
1 migraine
1 miller
4 mind
1 mindfulness
1 mini
1 minus
1 miracle
1 misconceptions
1 misguided
1 mising
55 miss
22 missed
2 misses
1 missherlots
10 missing
1 mistaking
1 mmhmm
1 mmm
1 mock
1 modern
1 moment
1 moments
9 mon
3 monday
1 money
1 months
48 more
2 morning
1 moron
1 mortar
38 most
1 mostly
1 mother
1 motivation
1 mouldy
2 move
1 moved
1 moves
1 movie
1 movies
2 moving
1 mta
1 mtgo
34 much
1 mugging
1 murder
3 murdered
1 murderer
2 music
9 must
1 muthafuckin
1 mutual
1 muwahahahhha
1 mwahahaha
57 my
1 myself
1 myth
1 na
1 nails
1 nanana
1 nananana

In [12]:
train_features = []
train_target = []
valid_features = []
valid_target = []
temp = int(0.8 * len(train_data_features))
for i in range(temp):
    train_features.append(train_data_features[i])
    train_target.append(train[4][i])
for i in range(temp, len(train_data_features)):
    valid_features.append(train_data_features[i])
    valid_target.append(train[4][i])
    
print(len(valid_features)+len(train_features))
print(len(train_data_features))

10475
10475


In [32]:
forest = RandomForestClassifier(n_estimators = 100, oob_score=True, random_state=3) 
forest = forest.fit( train_features, train_target )

In [33]:
result1 = forest.predict(valid_features)

In [34]:
print(len(result1))  
print(len(valid_target))
scores = 0
for i in range(len(result1)):
    if result1[i] == valid_target[i]:
        scores += 1
print(scores/len(valid_target))
    

2095
2095
0.7618138424821003


In [35]:
print(test.shape)
test.head()

(5267, 6)


,0,1,2,3,4,5
0,NaN,1,25,26,unknwn,@GMA My daughter @jordynking17 will be 14 on D...
1,NaN,2,13,13,unknwn,@moanajkidd The massacre on March 11 in Karm a...
2,NaN,3,2,2,unknwn,Happy Veteran day....may God continue 2 Bless ...
3,NaN,4,10,10,unknwn,"I miss old school Dashboard Confessional, taki..."
4,NaN,5,10,11,unknwn,@f_cancellara @Klitschko You weren't expecting...


In [36]:
clean_test_reviews = [] 
for i in range(0, len(test[5])):
    if( (i+1) % 1000 == 0 ):
        print("Review %d of %d\n" % (i+1, len(test[5])))
    clean_review = review_to_words( test[5][i], test[2][i], test[3][i] )
    clean_test_reviews.append(clean_review)

Review 1000 of 5267

Review 2000 of 5267

Review 3000 of 5267

Review 4000 of 5267

Review 5000 of 5267



In [37]:
print(clean_test_reviews)

['so', 'killed', 'day....may', 'fail,', 'good', 'chipped', 'win', "don't even feel like", 'cute...', 'havent', 'hee..', 'strength', 'totally just', 'not', 'be', 'pleasantly', 'good', 'celebration', 'won\\u2019t', 'saturday:(', 'so numerically', 'no', 'able', 'wanna', 'special', 'not', 'good', 'bad', 'sale', 'amazing', 'not', 'wow!', 'not', 'haven', 'shift', 'come', 'calm', '*hugz*', 'bestest', 'great', "didn't", 'will not', 'better', 'by', 'no', 'ok', 'ever', 'tense', 'so', 'accidentally', 'good', 'wanna', '!', 'enjoy abundant', 'stay', 'glad', 'clearance', 'come', 'lost', '1st', 'lure', 'no', 'ended in', 'looking', 'still', 'happy', 'cool', 'despair', 'scared', 'long time no', 'promoted', 'beautiful', 'updated', 'great', 'never', 'slowly', 'make', 'not', 'dead', 'still', "hadn't", 'darling', 'gorgeous', 'weird...', 'like', 'dun', 'unfortunate', 'can', 'really', 'refuse', 'miss', 'team...good', 'wtf!', 'very', 'don\\u2019t be', 'good', 'drunken', 'can', 'not', 'damn', 'so', 'have a gre

In [38]:
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

In [39]:
print(result)

['positive' 'negative' 'neutral' ..., 'negative' 'positive' 'negative']


In [41]:
# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test[1], "polarity":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "resultccc.tsv", index=False, quoting=3 )

In [67]:
ahaha = pd.read_csv("./resultbbb.tsv", delimiter="\t", quoting=3)
print(ahaha)

        id,polarity
0        1,positive
1        2,positive
2        3,positive
3        4,positive
4        5,negative
5        6,positive
6        7,positive
7        8,positive
8        9,negative
9       10,negative
10      11,negative
11      12,positive
12      13,positive
13      14,positive
14      15,negative
15      16,positive
16      17,positive
17      18,positive
18      19,negative
19      20,negative
20      21,negative
21      22,negative
22      23,positive
23      24,positive
24      25,positive
25      26,negative
26      27,positive
27      28,negative
28      29,positive
29      30,positive
...             ...
5237  5238,positive
5238  5239,negative
5239  5240,positive
5240  5241,positive
5241  5242,positive
5242  5243,negative
5243  5244,negative
5244  5245,positive
5245  5246,negative
5246  5247,positive
5247  5248,positive
5248  5249,negative
5249  5250,positive
5250  5251,negative
5251  5252,positive
5252  5253,positive
5253  5254,negative
5254  5255,negative
